In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/travel-insurance-prediction-data/TravelInsurancePrediction.csv")
df.head(5)

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
columns = df.columns
X = df[set(columns) - set(["Unnamed: 0", "TravelInsurance"])]
y = df["TravelInsurance"]

In [ ]:
df.describe()

In [ ]:
X


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [ ]:
X_test

In [ ]:
X_train.isna().sum() 

In [ ]:
X_train.size

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.histplot(data = X_train, x = "FamilyMembers")

In [ ]:
sns.histplot(data = X_train, x = "Age")

In [ ]:
sns.histplot(data = X_train, x = "AnnualIncome")

In [ ]:
columns

In [ ]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import make_column_transformer

trs = make_column_transformer(
    (MinMaxScaler(feature_range=(0, 1)),["AnnualIncome", "Age", "FamilyMembers"]),
    (OneHotEncoder(handle_unknown="ignore"), ["FrequentFlyer", "GraduateOrNot", "Employment Type", "EverTravelledAbroad"]),
#     sparse_threshold=1.0
)

X_normal_train = trs.fit_transform(X_train)

In [ ]:
X_normal_test = trs.fit_transform(X_test)

In [ ]:
print(X_normal_train)

In [ ]:
X_normal_train.shape

In [ ]:
X_train.shape

In [ ]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1)
#         tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

model.compile(
    loss = tf.keras.losses.mse,
    optimizer = tf.keras.optimizers.Adam(lr=0.01),
    metrics = ["accuracy"]
)

In [ ]:
callbacks = tf.keras.callbacks.EarlyStopping(monitor="accuracy", patience=3)
# tensorboard_callback = tf.keras.callbacks.TensorBoard("logs")
callbacks_2 = tf.keras.callbacks.TensorBoard(log_dir='logs',
                                 histogram_freq=0, 
                                 write_graph=True, 
                                 write_images=False,    
                                 update_freq='epoch', 
                                 profile_batch=2, 
                                 embeddings_freq=0,    
                                 embeddings_metadata=None, 
                                 )
history = model.fit(X_normal_train, y_train, epochs=20, callbacks=[callbacks, callbacks_2])
# print(history.history.keys())

In [ ]:
X_test

In [ ]:
model.evaluate(tf.keras.backend.cast_to_floatx(X_normal_test), y_test)

In [ ]:
print(history.history.keys())

In [ ]:
sns.lineplot(data = history.history["accuracy"])

In [ ]:
sns.lineplot(data = history.history["loss"])

In [ ]:
predictions = model.predict(tf.keras.backend.cast_to_floatx(X_normal_test))

In [ ]:
predictions